In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
torch.manual_seed(2020)
np.random.seed(2020)

In [3]:
{'lr': 0.000797977911440375,
 'layers': 3,
 'step_size': 15,
 'gamma': 0.6484984709565126,
 'bptt': 15,
 'dropout': 0.0294619119320717}

{'lr': 0.000797977911440375,
 'layers': 3,
 'step_size': 15,
 'gamma': 0.6484984709565126,
 'bptt': 15,
 'dropout': 0.0294619119320717}

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/penn.csv')

In [6]:
dataset.shape

(8757, 1045)

In [7]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,5,0,3,0,0,0,26,0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-01,4,3,0,7,0,0,0,8,0,...,0.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0
2,2018-01-01,5,6,0,1,1,0,2,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0


In [8]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

774

In [9]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 271)

In [10]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

257

In [11]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [12]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 271)
(2190, 271)


In [13]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 257])
torch.Size([2190, 13])
torch.Size([2190, 257])


In [14]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [15]:
bptt = 15

In [16]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [17]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([15, 13]), torch.Size([15, 257]), torch.Size([15, 257]))

In [18]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [19]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10, dropout=0, at_mat=None):
        super().__init__()
        
        # aggregation
        if at_mat != None:
            self.attachment_matrix = torch.nn.Parameter(at_mat)
            self.attachment_matrix.requires_grad = False
        else:
            self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
            self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers, dropout=dropout)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [20]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [21]:
def get_at_mat(targetColumns):
    comms = pd.read_csv('/home/urwa/Documents/side_projects/urban/UrbanTemporalNetworks/Data/ZonetoComm.csv')  
    communities = list(set(comms.start_community))

    mapping = dict(zip(comms.start_id, comms.start_community))
    comm_to_index = dict(zip(communities,range(len(communities))))
    col_to_index = dict(zip(targetColumns,range(len(targetColumns))))

    attach = torch.zeros(len(targetColumns), len(communities))

    for t_c in targetColumns:
        com = mapping[int(t_c)]
        x_i = col_to_index[t_c]
        y_i = comm_to_index[com]

        attach[x_i,y_i] = 1

    return attach

In [22]:
at_mat = get_at_mat(targetColumns)
at_mat.shape

torch.Size([257, 24])

In [23]:
layers = 3
communities = 24
network_size = len(targetColumns)
feat_size = len(features_cols)
dropout = 0.0294619119320717

model = LSTM(feat_size = feat_size, hidden_layer_size=communities,
             network_size=network_size, layers=layers,
            communities=communities, dropout=dropout, at_mat=at_mat).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.000797977911440375)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.648)

In [24]:
epochs = 100
best_r2 = 0

for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
#     if i%10 == 1:
    r2, rmse, mae = evaluate(model)
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

    if r2 > best_r2:
        best_r2 = r2
        torch.save(model.state_dict(), 'penn.pt')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')
print("bet_r2: ", best_r2)

epoch:   0 loss: 0.84347147 r2: 0.420 rmse: 9.387 mae: 0.971
epoch:   1 loss: 0.72685772 r2: 0.533 rmse: 7.561 mae: 0.890
epoch:   2 loss: 0.68050951 r2: 0.605 rmse: 6.403 mae: 0.862
epoch:   3 loss: 0.66250342 r2: 0.617 rmse: 6.195 mae: 0.847
epoch:   4 loss: 0.65180355 r2: 0.644 rmse: 5.760 mae: 0.830
epoch:   5 loss: 0.64066255 r2: 0.643 rmse: 5.780 mae: 0.829
epoch:   6 loss: 0.62875628 r2: 0.644 rmse: 5.767 mae: 0.837
epoch:   7 loss: 0.63417852 r2: 0.648 rmse: 5.699 mae: 0.837
epoch:   8 loss: 0.62599087 r2: 0.666 rmse: 5.410 mae: 0.806
epoch:   9 loss: 0.61837393 r2: 0.671 rmse: 5.334 mae: 0.807
epoch:  10 loss: 0.62578988 r2: 0.666 rmse: 5.405 mae: 0.814
epoch:  11 loss: 0.61956882 r2: 0.670 rmse: 5.345 mae: 0.803
epoch:  12 loss: 0.62543398 r2: 0.666 rmse: 5.409 mae: 0.813
epoch:  13 loss: 0.62031138 r2: 0.663 rmse: 5.449 mae: 0.814
epoch:  14 loss: 0.61905652 r2: 0.669 rmse: 5.354 mae: 0.806
epoch:  15 loss: 0.61735809 r2: 0.678 rmse: 5.211 mae: 0.800
epoch:  16 loss: 0.61590

In [25]:
evaluate(model)

(0.6974371056178693, 4.8986483, 0.7709814)

In [26]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

{'1': 0,
 '10': 4,
 '100': 0,
 '101': 4,
 '102': 23,
 '106': 19,
 '107': 1,
 '108': 3,
 '109': 5,
 '11': 3,
 '111': 19,
 '112': 9,
 '113': 1,
 '114': 1,
 '115': 18,
 '116': 22,
 '117': 16,
 '118': 5,
 '119': 14,
 '12': 1,
 '120': 22,
 '121': 8,
 '122': 4,
 '123': 3,
 '124': 4,
 '125': 1,
 '126': 14,
 '127': 22,
 '128': 22,
 '129': 23,
 '13': 1,
 '130': 4,
 '131': 4,
 '132': 4,
 '133': 3,
 '134': 4,
 '135': 8,
 '136': 10,
 '137': 0,
 '138': 0,
 '139': 4,
 '14': 3,
 '140': 15,
 '141': 15,
 '142': 12,
 '143': 12,
 '144': 1,
 '145': 7,
 '146': 7,
 '147': 14,
 '148': 1,
 '149': 3,
 '15': 8,
 '150': 3,
 '151': 12,
 '152': 22,
 '153': 10,
 '154': 3,
 '155': 3,
 '156': 11,
 '157': 23,
 '158': 1,
 '159': 14,
 '16': 8,
 '160': 23,
 '161': 0,
 '162': 0,
 '163': 0,
 '164': 0,
 '165': 3,
 '166': 12,
 '167': 14,
 '168': 14,
 '169': 14,
 '17': 9,
 '170': 0,
 '171': 8,
 '172': 6,
 '173': 23,
 '174': 10,
 '175': 8,
 '176': 5,
 '177': 13,
 '178': 3,
 '179': 7,
 '18': 10,
 '180': 4,
 '181': 19,
 '182': 2

In [27]:
# 20 comm
# 0.505

In [28]:
# 50 comm
# 